In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import os
from colour_mappings import line_colours, line_names
from graph_loading import load_graph

In [ ]:
G, pos, colors, weights, index_to_station_name, station_name_to_index = load_graph()

In [ ]:
# Basic Graph Properties
print("Basic Network Properties:")
print(f"Number of nodes (stations): {G.number_of_nodes()}")
print(f"Number of edges (connections): {G.number_of_edges()}")
print(f"Network density: {nx.density(G):.3f}")

In [ ]:
print("\nCentrality Analysis:")
# Degree centrality (stations with most connections)
degree_cent = nx.degree_centrality(G)
top_degree = sorted(degree_cent.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop 5 stations by degree centrality (most connections):")
for node, cent in top_degree:
    print(f"{index_to_station_name[node]}: {cent:.3f}")

In [ ]:
between_cent = nx.betweenness_centrality(G, weight='weight')
top_between = sorted(between_cent.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop 5 stations by betweenness centrality (important transfer points):")
for node, cent in top_between:
    print(f"{index_to_station_name[node]}: {cent:.3f}")

In [ ]:
print("\nShortest Paths Analysis:")
avg_path_length = nx.average_shortest_path_length(G, weight='weight')
print(f"Average shortest path length (in minutes): {avg_path_length:.2f}")

In [ ]:
diameter = nx.diameter(G, weight='weight')
print(f"Network diameter (longest shortest path in minutes): {diameter:.2f}")

In [ ]:
print("\nConnectivity Analysis:")
is_strongly_connected = nx.is_strongly_connected(G)
print(f"Is network strongly connected? {is_strongly_connected}")
if not is_strongly_connected:
    n_components = nx.number_strongly_connected_components(G)
    print(f"Number of strongly connected components: {n_components}")

In [ ]:
print("\nLine Analysis:")
colors = set(nx.get_edge_attributes(G, 'color').values())
print(f"Number of different lines: {len(colors)}")
for color in colors:
    edges = [(u,v) for (u,v,d) in G.edges(data=True) if d['color'] == color]
    print(f"Line {line_names[color]}: {len(edges)} connections")

In [ ]:
edge_betweenness = nx.edge_betweenness_centrality(G, weight='weight')
top_bottlenecks = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop 5 potential bottleneck connections:")
for (u, v, d), cent in top_bottlenecks:
    print(f"{index_to_station_name[u]} -> {index_to_station_name[v]}: {cent:.3f}")

In [ ]:
def stations_within_time(G, source, time_threshold):
    paths = nx.single_source_dijkstra_path_length(G, source, weight='weight')
    return sum(1 for t in paths.values() if t <= time_threshold)
print("\nStation Accessibility Analysis:")
central_stations = [node for node, cent in top_between[:3]]  # Using top 3 by betweenness
for station in central_stations:
    for threshold in [10, 20, 30]:  # minutes
        count = stations_within_time(G, station, threshold)
        print(f"From {index_to_station_name[station]}, {count} stations can be reached within {threshold} minutes")